# Creating Collections as Data Using Federated Queries / Single Author's Works (2)

## About the Notebook
This notebook demonstrates a streamlined pipeline for executing federated queries across multiple library catalogs. It simplifies the process of **retrieving and consolidating data about literary works from a specific author into a single dataset.** The final dataset is then exported as a CSV file for further analysis or use.

**[María de Zayas y Sotomayor](https://www.wikidata.org/wiki/Q2905689) (1590-1661)** was a popular novelist and playwright during the Spanish Golden Age. She is recognized for her witty and insightful novellas that often challenged societal norms and championed women's rights.

[![Launch](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/semanticnoodles/federated-cad/HEAD?filepath=notebooks/02-single-author/LOD-MariaZayas-Federated-CAD.ipynb)


<!--🔖 **How to Cite**: [![DOI](https://zenodo.org/badge/DOI_NUMBER.svg)](https://doi.org/DOI_NUMBER) fix once we get the CITATION.cff set up in the GitHub repo-->

---

## Getting Started
The code imports necessary libraries for data handling and database interaction, using [pandas](https://pypi.org/project/pandas/) and [SPARQLWrapper](https://pypi.org/project/SPARQLWrapper/#description). It then sets up a connection to the Wikidata database and defines a function called `execute_query` that can run queries and retrieve results in a convienent format. Essentially, it sets the stage to efficiently fetch and work with data from Wikidata.

In [ ]:
# Load modules
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Define the SPARQL endpoint
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

# Define a function to execute a query and return results as a list of dictionaries
def execute_query(query):
    sparql.setQuery(query)
    results = sparql.query().convert()
    return results["results"]["bindings"]

## Collating the Queries

This section of the code defines a Python dictionary called queries. This dictionary currently stores **three SPARQL queries**`*`, each designed to retrieve information from a different library catalog. The key value pairs on use are the following:

- `BNE`: Biblioteca Nacional de España (The Spanish National Library).
- `BNF`: Bibliothèque nationale de France (The National Library of France).
- `BVMC`: Biblioteca Virtual Miguel de Cervantes (The Miguel de Cervantes Virtual Library).

### General description
The overall goal of these queries is to find information about works by the author María de Zayas y Sotomayor. They achieve this by identifiying the author using their Wikidata identifier (wd:Q2905689). Then they retrieve details about each author's work, including author's name, edition, place of production/publication, year of publication and language on use identifier.

---

`*`Notebooks unpacking the creation of each of the following queries are available at [the source repository](https://github.com/hibernator11/federated-lod-queries).



In [ ]:
# Define queries
queries = {
    'BNE': """
        PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode
        WHERE {
            wd:Q2905689 wdt:P950 ?id .
            wd:Q2905689 rdfs:label ?author.  FILTER(LANG(?author) = "en").
            BIND(uri(concat("https://datos.bne.es/resource/", ?id)) as ?bneID)
            SERVICE <http://datos.bne.es/sparql> {
                ?bneID bne-def:OP5001 ?work .
                ?work rdfs:label ?workLabel .
                OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3003 ?placeOfProduction}
                OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3006 ?yearOfPublication}
                OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition dcterms:language ?langCode}
            }
        }
        LIMIT 1000
    """,
    'BNF': """
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
        PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        SELECT ?author ?expression ?title ?edition ?placeOfPublication ?yearOfPublication ?langCode WHERE {
            wd:Q2905689 wdt:P268 ?id
            BIND(uri(concat(concat("http://data.bnf.fr/ark:/12148/cb", ?id),"#about")) as ?author)
            SERVICE <http://data.bnf.fr/sparql> {
                ?expression <http://id.loc.gov/vocabulary/relators/aut> ?author .
                OPTIONAL {?expression dcterms:language ?langCode .}
                OPTIONAL {?expression dcterms:publisher ?edition .}
                ?manifestation rdarelationships:expressionManifested ?expression .
                ?manifestation dcterms:title ?title .
                ?manifestation dcterms:date ?yearOfPublication .
                OPTIONAL{ ?manifestation rdagroup1elements:placeOfPublication ?placeOfPublication .}
            }
        }
        LIMIT 1000
    """,
    'BVMC': """
        PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>

        SELECT ?author ?work ?workLabel ?placeOfProduction ?yearOfPublication ?langCode
        WHERE {
            wd:Q2905689 wdt:P2799 ?id .
            wd:Q2905689 rdfs:label ?author.  FILTER(LANG(?author) = "en").
            BIND(uri(concat("https://data.cervantesvirtual.com/person/", ?id)) as ?bvmcID)
            SERVICE <http://data.cervantesvirtual.com/openrdf-sesame/repositories/data> {
                ?work rdaw:author ?bvmcID .
                ?work rdfs:label ?workLabel .
                ?work rdaw:manifestationOfWork ?manifestation .
                ?work rdaw:expressionOfWork ?expression .
                OPTIONAL {?expression rdae:languageOfExpression ?language . ?language madsrdf:code ?langCode .}
                OPTIONAL {?manifestation rdam:placeOfProduction ?placeOfProduction .}
                OPTIONAL {?manifestation rdam:dateOfPublication ?dateOfPublication . BIND(REPLACE(str(?dateOfPublication), "https://data.cervantesvirtual.com/date/", "", "i") AS ?yearOfPublication) .}
            }
        }
        LIMIT 1000
    """
}

## Processing Queries Results

The function `process_query_results` transforms raw database query responses into a structured list of dictionaries. It uses a predefined mapping to standardize field names from different data sources and handles missing values by assigning them as `Unknown`. The cumulative structured list is then returned for further processing.

Once called the function, the remaining code chunks structure and combine data from multiple library catalogs into a single dataframe, displaying a preview and a summary of the results. Last, the dataframe is exported in .CSV format.

In [ ]:
# Function to process query results and return a list of dictionaries
def process_query_results(results, source):
    processed_data = []

    # Define the field name mapping depending on the source
    field_mapping = {
        'BNE': {
            'author': 'author',
            'work': 'work',
            'workLabel': 'workLabel',
            'edition': 'edition',
            'placeOfProduction': 'placeOfProduction',
            'yearOfPublication': 'yearOfPublication',
            'langCode': 'langCode'
        },
        'BNF': {
            'author': 'author',
            'work': 'expression',
            'workLabel': 'title',
            'edition': 'edition',
            'placeOfProduction': 'placeOfPublication',
            'yearOfPublication': 'yearOfPublication',
            'langCode': 'langCode'
        },
        'BVMC': {
            'author': 'author',
            'work': 'work',
            'workLabel': 'workLabel',
            'edition': 'edition',
            'placeOfProduction': 'placeOfProduction',
            'yearOfPublication': 'yearOfPublication',
            'langCode': 'langCode'
        }
    }

    # Map the query result fields to the corresponding output fields
    for result in results:
        # For each source, apply the field mapping
        author = result.get(field_mapping[source]['author'], {}).get('value', None)
        work = result.get(field_mapping[source]['work'], {}).get('value', None)
        workLabel = result.get(field_mapping[source]['workLabel'], {}).get('value', None)  # Removed comma here
        edition = result.get(field_mapping[source]['edition'], {}).get('value', None)  # Removed comma here
        placeOfProduction = result.get(field_mapping[source]['placeOfProduction'], {}).get('value', None)
        yearOfPublication = result.get(field_mapping[source]['yearOfPublication'], {}).get('value', None)
        langCode = result.get(field_mapping[source]['langCode'], {}).get('value', None)

        # Handle missing or empty fields
        placeOfProduction = placeOfProduction if placeOfProduction else 'Unknown'
        yearOfPublication = yearOfPublication if yearOfPublication else 'Unknown'
        langCode = langCode if langCode else 'Unknown'

        # Append the processed result to the list
        processed_data.append({
            'source': source,
            'author': author,
            'work': work,
            'workLabel': workLabel,
            'edition': edition,
            'placeOfProduction': placeOfProduction,
            'yearOfPublication': yearOfPublication,
            'langCode': langCode,
        })

    return processed_data

In [ ]:
# Create an empty list to collect the queries results
queries_results = []

# Execute each query, process the results, and extend to the queries_results list
for source, query in queries.items():
    results = execute_query(query)
    processed_data = process_query_results(results, source)
    queries_results.extend(processed_data)

# Create a dataframe from the collected results
df_combined = pd.DataFrame(
    queries_results,
    columns=[
        "source",
        "author",
        "work",
        "workLabel",
        "edition",
        "placeOfProduction",
        "yearOfPublication",
        "langCode",
    ],
)

# Preview the results
df_combined.head(10)

,source,author,work,workLabel,edition,placeOfProduction,yearOfPublication,langCode
0,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/bimo0000126542,Madrid,1980,http://id.loc.gov/vocabulary/languages/spa
1,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/bimo0001593058,Barcelona,[1973],http://id.loc.gov/vocabulary/languages/spa
2,BNE,María de Zayas,https://datos.bne.es/resource/XX2823015,El juez de su causa,https://datos.bne.es/resource/bimo0001593059,Madrid,1943,http://id.loc.gov/vocabulary/languages/spa
3,BNE,María de Zayas,https://datos.bne.es/resource/XX2823015,El juez de su causa,https://datos.bne.es/resource/bimo0001593060,Madrid,1932,http://id.loc.gov/vocabulary/languages/spa
4,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/bimo0001593061,París,1847,http://id.loc.gov/vocabulary/languages/spa
5,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/bimo0001593063,Barcelona,[1973],http://id.loc.gov/vocabulary/languages/spa
6,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/bimo0001593064,Madrid,[1968],http://id.loc.gov/vocabulary/languages/spa
7,BNE,María de Zayas,https://datos.bne.es/resource/XX2931426,Aventurarse perdiendo,https://datos.bne.es/resource/bimo0001593221,Barcelona,[1940],http://id.loc.gov/vocabulary/languages/spa
8,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/Mimo0001382849,[Barcelona],[1999],http://id.loc.gov/vocabulary/languages/spa
9,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/Mimo0001654380,Madrid,[2000],http://id.loc.gov/vocabulary/languages/spa


In [ ]:
# Describe the results
df_combined.describe()

,source,author,work,workLabel,edition,placeOfProduction,yearOfPublication,langCode
count,147,147,147,147,58,147,147,147
unique,3,2,91,75,54,46,75,9
top,BNE,María de Zayas,https://datos.bne.es/resource/XX2237525,Novelas amorosas y ejemplares,https://datos.bne.es/resource/a4523833,Madrid,Unknown,http://id.loc.gov/vocabulary/languages/spa
freq,75,106,41,42,2,29,24,52


In [ ]:
# Export the resulting dataframe in .CSV file format
df_combined.to_csv('df-maria-zayas.csv', index=False)